In [1]:
from fynesse import access, assess, address

from functools import partial
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tabulate import tabulate

In [2]:
from config import MODELS, EVENTS, SEEDS

## Utilities

In [3]:
from sklearn.model_selection import GridSearchCV

def print_grid_search_results(cv_results_):
    print(tabulate(np.column_stack([cv_results_['params'], cv_results_['mean_test_score'], cv_results_['rank_test_score']]), headers=['params', 'score', 'rank']))

In [4]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def evaluate(clf, X_test, y_test):
    accuracy = accuracy_score(y_test, clf.predict(X_test))
    f1 = f1_score(y_test, clf.predict(X_test), average='macro')
    precision = precision_score(y_test, clf.predict(X_test), average='macro')
    recall = recall_score(y_test, clf.predict(X_test), average='macro')

    return np.array([accuracy, f1, precision, recall])

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

def evaluate_feature(master_dict, featured_events):

    scores, scores_unseen = [], []

    for i, seed in enumerate(SEEDS):
        X = address.make_features(master_dict, featured_events, n_bins=32, models=MODELS, n_samples=500)
        scaler = StandardScaler().fit(X)
        X = scaler.transform(X)

        X, X_test_unseen = address.seed_split(X, seed_idx=i, n_seeds=len(SEEDS))

        le = LabelEncoder().fit(MODELS)

        y = list()
        for model in MODELS:
            y += [model] * 400
        y = le.transform(y)

        y_test_unseen = list()
        for model in MODELS:
            y_test_unseen += [model] * 100
        y_test_unseen = le.transform(y_test_unseen)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

        knn = KNeighborsClassifier(n_neighbors=10,
                                p=1,
                                weights='uniform')
        
        knn.fit(X_train, y_train)

        scores.append(evaluate(knn, X_test, y_test))
        scores_unseen.append(evaluate(knn, X_test_unseen, y_test_unseen))

    scores = np.column_stack(scores)
    scores_unseen = np.column_stack(scores_unseen)
    
    return scores, scores_unseen

## Feature Construction

In [6]:
# load master dictionary for everything

master_dict = assess.eat_pickle(f'./data/pickle/master.pickle')

In [11]:
results = []

for featured_events in combinations(EVENTS, 2):
    row = [featured_events]
    scores, scores_unseen = evaluate_feature(master_dict, featured_events)
    
    row += [np.mean(scores, axis=1)[0], np.mean(scores_unseen, axis=1)[0]]
    results.append(row)

/Users/kate/anaconda3/envs/diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kate/anaconda3/envs/diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kate/anaconda3/envs/diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [12]:
print(tabulate(results, headers=['events', 'accuracy', 'unseen accuracy']))

events                                                                                      accuracy    unseen accuracy
----------------------------------------------------------------------------------------  ----------  -----------------
('instructions', 'cache-misses')                                                            0.574              0.311333
('instructions', 'cache-references')                                                        0.546667           0.328667
('instructions', 'L1-dcache-load-misses')                                                   0.571333           0.317333
('instructions', 'L1-dcache-loads')                                                         0.553333           0.326
('instructions', 'L1-dcache-stores')                                                        0.569333           0.318667
('instructions', 'LLC-load-misses')                                                         0.549333           0.314667
('instructions', 'LLC-loads')              

## Feature Importance